In [ ]:
import hdbcli
import hana_ml

print("Modules imported successfully!")

In [51]:
from hdbcli import dbapi
import hana_ml.dataframe as dataframe
from gen_ai_hub.proxy.langchain.openai import OpenAIEmbeddings
from gen_ai_hub.proxy.core.proxy_clients import get_proxy_client
from langchain.chains import RetrievalQA
from langchain.text_splitter import CharacterTextSplitter
from langchain.text_splitter import TokenTextSplitter
from langchain.document_loaders import TextLoader
from langchain.document_loaders import PyPDFLoader
from langchain.document_loaders import UnstructuredExcelLoader
from langchain_community.vectorstores.hanavector import HanaDB

In [50]:
#This script is used to get the access token for the BTP LLM API
import json
import requests

with open("irpa-r998-genaixl-sk.json", "r") as key_file:
    svcKey = json.load(key_file)
authUrl = svcKey["url"]
clientid = svcKey["clientid"]
clientsecret = svcKey["clientsecret"]
apiUrl = svcKey["serviceurls"]["AI_API_URL"]

# request token
params = {"grant_type": "client_credentials" }
resp = requests.post(f"{authUrl}/oauth/token",
                    auth=(clientid, clientsecret),
                    params=params)

BtpLlmApiUrl = apiUrl
BtpLlmAccessToken = resp.json()["access_token"]

In [ ]:
# Set the environment variables for the BTP LLM API
import os 

# Define Keys HERE
env_vars = {    
 "AICORE_AUTH_URL": authUrl,
 "AICORE_CLIENT_ID": clientid,
 "AICORE_CLIENT_SECRET": clientsecret,
 "AICORE_RESOURCE_GROUP": "default",
 "AICORE_BASE_URL": apiUrl
}

os.environ.update(env_vars)

In [ ]:
from gen_ai_hub.proxy.native.openai import chat

messages = [ {"role": "user", "content": "Could you please clasify the data based on same category?"} ]

kwargs = dict(model_name='gpt-4o-mini', messages=messages)
response = chat.completions.create(**kwargs)
import json
response_dict = json.loads(response.json())
message_value = response_dict['choices'][0]['message']['content']
message_value

In [ ]:
from langchain.document_loaders import UnstructuredExcelLoader

# Load Excel files
loaders = [
    #UnstructuredExcelLoader("ITCdataset.xlsx"),
    UnstructuredExcelLoader("ITCdatamasked.xlsx"),
]

docs = []
for loader in loaders:
    docs.extend(loader.load())

print("Documents loaded successfully:", len(docs))

In [ ]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

# Modify the chunk_size and chunk_overlap parameters
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1500,  # You can change this value
    chunk_overlap=150  # You can change this value
)

splits = text_splitter.split_documents(docs)
len(splits)  # Output the number of chunks

In [ ]:
# Split
from langchain.text_splitter import RecursiveCharacterTextSplitter
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size = 1500,
    chunk_overlap = 150
)

In [ ]:
splits = text_splitter.split_documents(docs)
print(splits)

In [ ]:
len(splits)
print (len(splits))

In [ ]:
from gen_ai_hub.proxy.native.openai import embeddings

response = embeddings.create(
    input="Classification planning is complex, and people struggle to manage classification and grouping of similar items to find best vendor",
    model_name="text-embedding-ada-002"
)
print(response.data)

In [ ]:
len(response.data[0].embedding)
print(len(response.data[0].embedding))

In [ ]:
embedding_model = OpenAIEmbeddings(proxy_model_name='text-embedding-ada-002')

In [ ]:
sentence1 = "classification is complex to group the similar data"
sentence2 = "compare the prices and suggest the best vendoe"
sentence3 = "create a centrlized material code for all the uncoded data"

In [ ]:
from gen_ai_hub.proxy.langchain.openai import ChatOpenAI
from gen_ai_hub.proxy.core.proxy_clients import get_proxy_client

proxy_client = get_proxy_client('gen-ai-hub')
chat_llm = ChatOpenAI(proxy_model_name='gpt-4o-mini', proxy_client=proxy_client, temperature=0.0)

In [ ]:
embedding1 = embedding_model.embed_query(sentence1)
embedding2 = embedding_model.embed_query(sentence2)
embedding3 = embedding_model.embed_query(sentence3)

In [ ]:
import numpy as np

In [ ]:
np.dot(embedding1, embedding2)

In [ ]:
np.dot(embedding1, embedding3)

In [ ]:
np.dot(embedding2, embedding3)

In [ ]:
# Get the HANA Cloud username from environment variables
HANA_USER_VDB = 'DBADMIN'
# Get the HANA Cloud password from environment variables

HANA_PASSWORD_VDB = 'Qwerty@0303'
# Get the HANA Cloud host from environment variables - You need to configure this with your instance details
#Hint-  This is you SQL end point. But you need to remove ":443" from the SQL endpoint as port is already hardcoded in below cell
HANA_HOST = '653333c1-1c26-4c11-afc3-0b2962e0533f.hana.trial-us10.hanacloud.ondemand.com'

In [ ]:
# Use connection settings from the environment
connection = dbapi.connect(
    address=HANA_HOST,
    port=443,
    user=HANA_USER_VDB,
    password=HANA_PASSWORD_VDB,
    encrypt='true',
    autocommit=True
)

In [ ]:
# Connection Context
conn = dataframe.ConnectionContext(
    address=HANA_HOST,  
    port=443,
    user=HANA_USER_VDB,
    password=HANA_PASSWORD_VDB,
    encrypt='true',
    autocommit=True
)

In [ ]:
from gen_ai_hub.proxy.langchain.openai import ChatOpenAI
from gen_ai_hub.proxy.core.proxy_clients import get_proxy_client

proxy_client = get_proxy_client('gen-ai-hub')
chat_llm = ChatOpenAI(proxy_model_name='gpt-4o-mini', proxy_client=proxy_client, temperature=0.0)

In [ ]:
db = HanaDB(
    embedding=embedding_model,
    connection=connection,
    table_name="SAP_INVENT_ITC"
    # vector_column_length=1536
)

# # Delete already existing documents from the table
db.delete(filter={})

# # add the loaded document chunks 
db.add_documents(splits)

In [ ]:
# take a look at the table
hdf = conn.sql(''' SELECT "VEC_TEXT", "VEC_META", TO_NVARCHAR("VEC_VECTOR") AS "VEC_VECTOR" FROM "SAP_INVENT_ITC" ''')
df = hdf.head(10).collect()
df

In [ ]:
retriever = db.as_retriever()

In [ ]:
from langchain_core.prompts import ChatPromptTemplate
prompt = ChatPromptTemplate.from_template("""Provide answers based on context provided:

<context>
{context}
</context>

Question: {input}""")

In [ ]:
from langchain.chains import create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain

document_chain = create_stuff_documents_chain(chat_llm, prompt)
retrieval_chain = create_retrieval_chain(retriever, document_chain)

In [69]:
response = retrieval_chain.invoke({"input": "can you clasify the given data into five columns HSNcode, Vendor code, vendor name and short text. Then create a unique short material code with not more than ten charactors based on the short text and hsn code. "})
print(response["answer"])

Based on the provided context, here is the classification of the data into five columns: HSN Code, Vendor Code, Vendor Name, Short Text, and a unique Short Material Code. The Short Material Code is created by combining the first few characters of the Short Text and the HSN Code, ensuring it does not exceed ten characters.

| HSN Code      | Vendor Code   | Vendor Name | Short Text                          | Short Material Code |
|---------------|---------------|-------------|-------------------------------------|----------------------|
| 8473 30 99    | 2699386794    | Dell        | Dell Thunderbolt Dock - WD22TB4    | DTDWD22TB4           |
| 4818 10 00    | 3316872936    | Scott       | Scott Slim roll towel               | SRTOWEL              |
| 3401 30 90    | 3316872936    | Nimlyle     | Nimlyle Floor Cleaner               | NFCLEANER            |
| 3402 20 10    | 3316872936    | Savlon      | Savlon Handwash pump type           | SHANDWASH            |
| 3808 94 00    | 33168

In [54]:
generated = retrieval_chain.invoke({"input": "Can you classify the given data based on their similarities"})
print(generated["answer"])

Based on the provided context, the data can be classified into two main categories: **Product Information** and **Vendor Information**. 

### 1. Product Information
This category includes details about various products, which can be identified using the following fields:
- **Description**: The name or description of the product.
- **HSN Code**: The Harmonized System of Nomenclature code for classification.
- **Material Group**: The group to which the material belongs.

#### Examples:
- **PR_WAVE_SHOT_DSFT_PCK(PH-II)**: 
  - HSN Code: 4911 99 90
  - Material Group: AU
  - Net Price: 112488.5 INR
  - Effective Price: 132736.4 INR

- **Allyl Cyclo Hexyl Propionate**: 
  - HSN Code: 2916 20 00
  - Material Group: KG
  - Net Price: 6500 INR
  - Effective Price: 3250 INR

### 2. Vendor Information
This category includes details related to vendor ranking, which can be assessed using:
- **Net Price**: The base price of the product.
- **Effective Price**: The final price after any adjustments.


In [ ]:
generated = retrieval_chain.invoke({"input": "Can you suggest a best vendor and their name to procure the paper?"})
print(generated["answer"])

In [65]:
generated = retrieval_chain.invoke({"input": "can you suggest a top 3 best vendor to procure a garlic or similar product like lasun and provide pros and cons of each vendor in one line and give rank to vendor"})
print(generated["answer"])

Based on the provided context, here are the top 3 vendors for procuring garlic (lasun) or similar products:

1. **Vendor: ZZZ 9287663684**
   - **Pros:** Offers a large quantity (300 KG) at a competitive price (1400 INR).
   - **Cons:** Limited variety of garlic products available.
   - **Rank:** 1

2. **Vendor: ZZZ 3448954652**
   - **Pros:** Provides a diverse range of products including garlic at reasonable prices.
   - **Cons:** Smaller quantities available compared to other vendors.
   - **Rank:** 2

3. **Vendor: ZZZ 1702563974**
   - **Pros:** Offers garlic at a lower price point (105 INR for 750 KG).
   - **Cons:** Limited product range and higher minimum order quantities.
   - **Rank:** 3


In [66]:
generated = retrieval_chain.invoke({"input": "Can you suggest top 3 best vendor to procure ballpen and give rank to each vendor?"})
print(generated["answer"])

Based on the provided context, here are the top 3 vendors for procuring ball pens, ranked by the quantity and variety of ball pens available:

1. **Vendor: SRE CONSMOTH**
   - **Rank: 1**
   - **Details:**
     - Offers a wide variety of ball pens including:
       - BALL PEN BLUE (FOR WORKERS) - 2250 EA
       - BALL PEN BLUE - 1000 EA
       - GELL PEN BLACK - 100 EA
       - GELL PEN RED - 50 EA
       - GELL PEN BLUE - 150 EA
     - Total Quantity: 3600 EA

2. **Vendor: BLR CONSMSTN**
   - **Rank: 2**
   - **Details:**
     - Offers several types of pens including:
       - Classmate hook pen (blue) - 250 EA
       - RED PEN - 25 EA
       - Uniball eye Fine (Block PEN) - 50 EA
     - Total Quantity: 325 EA

3. **Vendor: AHM CONSMOTH**
   - **Rank: 3**
   - **Details:**
     - Offers a limited selection of ball pens:
       - Octen Gel Ballpen-BL - 250 EA
     - Total Quantity: 250 EA

These rankings are based on the quantity of ball pens available from each vendor, with SRE CONSMO

In [64]:
generated = retrieval_chain.invoke({"input": "I want to buy a ballpen in bulk order. can you suggest a vendor who can give best offer for bulk order with vendor code. also suggest some alternate vendor code for negotiation. give rank for each vendor based on the lowest unit price"})
print(generated["answer"])

Based on the provided context, here are the details for ballpen vendors along with their vendor codes and rankings based on the lowest unit price:

### Recommended Vendor for Bulk Order:
1. **Vendor Code: ZZZ 5813736581**
   - **Description:** BALL PEN BLUE (FOR WORKERS)
   - **Unit Price:** INR 32.95
   - **Quantity Available:** 2250 EA
   - **Total Price:** INR 92.75

### Alternate Vendors for Negotiation:
2. **Vendor Code: YYY 9848147211**
   - **Description:** Pen (Ball) Classmate
   - **Unit Price:** INR 125
   - **Quantity Available:** 25 NO
   - **Total Price:** INR 700

3. **Vendor Code: ZZZ 6599853239**
   - **Description:** RED PEN
   - **Unit Price:** INR 42.5
   - **Quantity Available:** 25 EA
   - **Total Price:** INR 250.8

4. **Vendor Code: ZZZ 6599853239**
   - **Description:** Classmate hook pen (blue)
   - **Unit Price:** INR 42.5
   - **Quantity Available:** 250 EA
   - **Total Price:** INR 2507.5

### Ranking Based on Lowest Unit Price:
1. **Vendor Code: ZZZ 5813736

In [60]:
generated = retrieval_chain.invoke({"input": "can you clasify the data based on codified product and non codified products"})
print(generated["answer"])

Based on the provided context, we can classify the data into codified and non-codified products as follows:

### Codified Products
These products have a specific HSN (Harmonized System of Nomenclature) code associated with them, indicating they are standardized and classified for trade purposes.

1. **Dell Thunderbolt Dock - WD22TB4**
   - HSN Code: 8473 30 99
   - Net Price: 110500 INR
   - Effective Price: 130390 INR

2. **Scott Slim Roll Towel**
   - HSN Code: 4818 10 00
   - Net Price: 2700 INR
   - Effective Price: 38232 INR

3. **Nimlyle Floor Cleaner**
   - HSN Code: 3401 30 90
   - Net Price: 350 INR
   - Effective Price: 6608 INR

4. **Savlon Handwash Pump Type**
   - HSN Code: 3402 20 10
   - Net Price: 400 INR
   - Effective Price: 3304 INR

5. **Domex Toilet Cleaner**
   - HSN Code: 3808 94 00
   - Net Price: 415 INR
   - Effective Price: 7345.5 INR

6. **Lizol 500 ml**
   - HSN Code: 3401 30 90
   - Net Price: 465 INR
   - Effective Price: 4938.3 INR

7. **Vim Liquid 500 m

In [55]:
generated = retrieval_chain.invoke({"input": "what u understand from data we provided"})
print(generated["answer"])

The provided data appears to be a structured list of transactions or orders, likely from a business or inventory management system. Each entry contains various fields that provide detailed information about the items or services being ordered. Here are some key observations:

1. **Transaction Details**: Each entry includes a unique identifier (e.g., numbers like 2402, 8531, etc.), dates (e.g., 2024-01-09), quantities (e.g., 5 EA), and pricing information (e.g., 35000 INR).

2. **Item Descriptions**: The data includes descriptions of the items or services, such as "GFK Red Firm FT MDPPCT-CC" or "AIRTEL 4G DATA CARD SERVICE_EGAU_Jan'24," indicating a variety of products and services.

3. **Categories**: The entries are categorized under different types, such as "CONSMOTH" (likely consumer goods) and "SERVMISC" (miscellaneous services), suggesting a diverse range of offerings.

4. **Pricing Information**: Each entry includes pricing details, including unit prices and total amounts, which 

In [63]:
generated = retrieval_chain.invoke({"input": "Can you suggest a best tea?"})
print(generated["answer"])

Based on the context provided, two notable options for tea are:

1. **Tetley Green Tea (100s)** - Priced at 2,975 INR for a pack of 100, it is a popular choice known for its quality and flavor.

2. **Tata Tea Bag (100 pic)** - Priced at 1,000 INR for a pack of 100, this is another well-regarded option that offers good value.

Both options are well-reviewed and widely consumed, making them excellent choices for tea lovers. If you prefer green tea, Tetley would be the best pick, while Tata Tea is a great option for traditional tea.


In [67]:
generated = retrieval_chain.invoke({"input": "What's the cost of Diethyl Malonate?"})
print(generated["answer"])

The cost of Diethyl Malonate is 3,250 INR for 5 KG.
